# SWEEP Operator

Where $A = (a_{ij})$ is a $n\times n$ square matrix,
the SWEEP operator on $A$ and $k$ is defined as $\textrm{SWEEP}(A, k) = \tilde {A}$, where

$\tilde{a}_{ij} = a_{ij} − \dfrac{a_{ik}a_{kj}}{a_{kk}}$, for all $i,j \neq k$,

$\tilde{a}_{kj} = \dfrac{a_{kj}}{a_{kk}}$, for all $j \neq k$,

$\tilde{a}_{ik} = \dfrac{a_{ik}}{a_{kk}}$, for all $i \neq k$,

$\tilde{a}_{kk} = - \dfrac{1}{a_{kk}}$.


In [1]:
import numpy as np

def sweep(A, k):
    """
    Perform a SWEEP operation on A with the pivot element A[k,k].
    
    :param A: a square matrix.
    :param k: the pivot element is A[k, k].
    :returns a swept matrix. Original matrix is unchanged.
    """
    n = A.shape[0]
    if A.shape != (n, n):
        raise ValueError('A is not a square array.')
    if k >= n or k < 0:
        raise IndexError('k is not a valid row index for pivot element.')
        
    #  Fill with the general formula
    B = A - np.outer(A[:, k], A[k, :]) / A[k, k]
    
    # Modify the k-th row and column
    B[k, :] = A[k, :] / A[k, k]
    B[:, k] = A[:, k] / A[k, k]
    
    # Modify the pivot
    B[k, k] = -1 / A[k, k]
    return B

## Apply repeatedly

Suppose we split a $n\times n$ matrix $A$ into four blocks
$$A = \begin{bmatrix} A_{11} & A_{12} \\ A_{21} & A_{22} \end{bmatrix},$$ 
where $A_{11}$ is a $m\times m$ matrix.

Using SWEEP operator repeatedly with $k=1, 2, \ldots, m$, we get
$$\textrm{SWEEP}(A, 1\cdots m) = \begin{bmatrix} -A_{11}^{-1} & A_{11}^{-1}A_{12} \\ A_{21}A_{11}^{-1} & A_{22} - 
A_{21}A_{11}^{-1}A_{12} \end{bmatrix}.$$

As a special case, $\textrm{SWEEP}(A, 1\cdots n) = -A^{-1}$.

In [2]:
# Use our sweep function.
A = np.array([[1,2,3],[7,11,13],[17,21,23]], dtype=float)

# Perform sweep operator repeatedly.
det_A = 1
A_swp = A.copy()
for k in (0, 1, 2):
    det_A *= A_swp[k, k]
    A_swp = sweep(A_swp, k)
    
print 'A_swp:\n', A_swp
print 'Determinant of A:', det_A

A_swp:
[[-1.    0.85 -0.35]
 [ 3.   -1.4   0.4 ]
 [-2.    0.65 -0.15]]
Determinant of A: -20.0


## Solving the least square

In linear regression $Y = X^{T}\beta + \epsilon$, where $\epsilon_i \sim N(0, \sigma^2)$.

When solving the least square estimator
$$\hat{\beta} = \arg\min_{\beta} |Y - X^{T}\beta|^{2} = (X^{T}X)^{-1}X^{T}Y,$$
we construct a matrix $Z = \begin{bmatrix} X & Y \end{bmatrix}$,

$$A= Z^{T}Z = \begin{bmatrix} X^{T}X & X^{T}Y \\ Y^{T}X & Y^{T}Y \end{bmatrix}.$$

Using SWEEP operator repeatedly with $k=1,2,\ldots, p$, 
$$\textrm{SWEEP}(A, 1\cdots p) = \begin{bmatrix} -(X^{T}X)^{-1} & (X^{T}X)^{-1}X^{T}Y \\ Y^{T}X(X^{T}X)^{-1} & Y^{T}Y - Y^{T}X(X^{T}X)^{-1}X^{T}Y \end{bmatrix} = \begin{bmatrix} -\dfrac{\mathrm{Var}[\hat{\beta}]}{\sigma^2} & \hat{\beta} \\ \hat{\beta}^{T} & \textrm{RSS} \end{bmatrix},$$
where $\textrm{RSS} = Y^{T}Y - \hat{Y}^{T}\hat{Y}$ is the residual sum of squares.

In [3]:
#======================================
# Generate a testing data.
#======================================
n, p = 100, 5

# Generate a random n-by-p data matrix.
X = np.random.normal(0, 1, (n, p))

# Assume the real coefficients are 1 ... p. Intercept is 0.
# This is the ground-truth we want to evaluate our code against.
beta = np.array(range(1, p+1))

# Synthesis the output Y.
Y = np.dot(X, beta)

#======================================
# Solve by scipy.
#======================================
from scipy import linalg
coef, resid, rank, sigma = linalg.lstsq(X, Y)
print '----- SCIPY -----'
print '[scipy] coefficients:', coef.round(6)
print '[scipy]      residue:', resid.round(6)

#======================================
# Solve by sweep.
#======================================
# Stack an additional n-by-1 ones vector for solving intercepts.
Z = np.hstack((np.ones(n).reshape((n, 1)), X, Y.reshape((n, 1))))
A = np.dot(Z.T, Z)

S = A
for k in range(p+1): # +1 because we added one intercept column.
    S = sweep(S, k)
    
beta = S[:p+1, -1]
rss = S[-1, -1]
print '----- SWEEP -----'
print '[sweep] coefficients:', beta.round(6)
print '[sweep]     residure:', rss.round(6)

----- SCIPY -----
[scipy] coefficients: [ 1.  2.  3.  4.  5.]
[scipy]      residue: 0.0
----- SWEEP -----
[sweep] coefficients: [-0.  1.  2.  3.  4.  5.]
[sweep]     residure: -0.0
